In [ ]:
import pandas as pd 
import numpy as np
user_data = pd.read_table('./users.dat', sep = '::', names = ['user_id', 'gender', 'age', 'occupation', 'time'])
inter_data = pd.read_table('./ratings.dat', sep = '::', names = ['user_id', 'item_id', 'ratings', 'timestamp'])
data = pd.merge(inter_data, user_data)
from sklearn.preprocessing import LabelEncoder
lbe = LabelEncoder()
cnt = data['user_id'].value_counts()
users_50 = list(cnt[cnt>=50].index.tolist())
data = data.loc[data['user_id'].isin(users_50), :]
cnt = data['item_id'].value_counts()
items_10 = list(cnt[cnt>=10].index.tolist())
data = data.loc[data['item_id'].isin(items_10), :]
from sklearn.preprocessing import LabelEncoder
lbe = LabelEncoder()
data['user_id'] = lbe.fit_transform(data['user_id'])
data['item_id'] = lbe.fit_transform(data['item_id'])
data.loc[data['gender']=='F','gender']=0
data.loc[data['gender']=='M','gender']=1
data.reset_index(inplace=True, drop=True)
data.to_hdf('./ml-1m.h5', key = '50core')
n_users = data['user_id'].drop_duplicates()  #获取index
n_items = data['item_id'].drop_duplicates() #获取columns

df_ = pd.DataFrame(index=n_users, columns=n_items) #新定义一个数据框df
for i in range(len(data)):
    row_i = data.loc[[i]]
    user_id = (row_i['user_id'].tolist())[0]
    items_name = (row_i['item_id'].tolist())[0]
    rating = (row_i['ratings'].tolist())[0]
    df_.at[user_id,items_name] = rating
df = df_.fillna(0)
df['user_id'] = df.index
df.index.name = 'user'
new_data = pd.merge(df.copy(),data.copy(), on = 'user_id')
new_data.to_hdf('./ml-1m-intermat.h5', key = 'intermat')